En este notebook se consolida un webscraping de Yapo, donde paginamos cada uno de todos sus autos esto nos sirve como forma de obtener un espectro de todo Yapo.

In [1]:
import re
import csv
import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv, datetime, time
import requests
from bs4 import BeautifulSoup
import time
from collections import Counter
import os
import pandas as pd
import numpy as np

Lo bueno de este codigo es que muy manejable puedes colocar la pagina donde quieres inciar, pagina finasl donde quieres ir, y como es largo tenemos un batch de guardar cada 100 autos.

In [ ]:


# ===== CONFIG =====
OUTPUT_CSV = "yapo_autos_1.csv"
BASE_URL = "https://www.yapo.cl/autos-usados."
MAX_PAGES = 1313
BATCH_SIZE = 100  # cada 100 autos guarda
START_PAGE = 1   # <-- pon un número o "auto" para reanudar desde CSV

options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

def limpiar_numero(texto):
    if not texto:
        return 0
    return int(re.sub(r"[^\d]", "", texto))

def guardar_csv(rows, append=True):
    """Guarda registros en el CSV"""
    if not rows:
        return
    file_exists = os.path.exists(OUTPUT_CSV)
    mode = "a" if append else "w"
    with open(OUTPUT_CSV, mode, newline="", encoding="utf-8") as f:
        fieldnames = [
            "timestamp", "pagina", "titulo_auto", "vendedor",
            "es_automotora", "total_autos_vendedor", "plan_pagado",
            "precio", "anio", "km", "ubicacion", "fecha_publicacion", "link"
        ]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists or not append:
            writer.writeheader()
        writer.writerows(rows)
    print(f"💾 Guardados {len(rows)} registros en {OUTPUT_CSV}")

def detectar_ultima_pagina_csv():
    """Detecta la última página guardada en el CSV para reanudar"""
    if not os.path.exists(OUTPUT_CSV):
        return None
    try:
        with open(OUTPUT_CSV, "r", encoding="utf-8") as f:
            rows = list(csv.DictReader(f))
            if not rows:
                return None
            ultima = rows[-1].get("pagina")
            return int(ultima) if ultima and ultima.isdigit() else None
    except:
        return None

try:
    # Determinar página de inicio
    if START_PAGE == "auto":
        ultima = detectar_ultima_pagina_csv()
        if ultima:
            start_page = ultima + 1
            print(f"🔄 Reanudando desde la página {start_page}")
        else:
            start_page = 1
    else:
        start_page = START_PAGE

    all_rows = []

    for page in range(start_page, MAX_PAGES + 1):
        try:
            url = f"{BASE_URL}{page}"
            print(f"🌐 Página {page}/{MAX_PAGES}: {url}")
            driver.get(url)

            try:
                autos = wait.until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.d3-ad-tile"))
                )
            except:
                print(f"⚠️ No se encontraron autos en la página {page}, deteniendo.")
                break

            autos_info = []
            for auto in autos:
                try:
                    titulo = auto.find_element(By.CSS_SELECTOR, "span.d3-ad-tile__title").text.strip()
                except:
                    titulo = "N/A"

                try:
                    vendedor = auto.find_element(By.CSS_SELECTOR, "div.d3-ad-tile__seller span").text.strip()
                except:
                    vendedor = "N/A"

                tiene_pro = len(auto.find_elements(By.CSS_SELECTOR, "div.d3-ad-tile__seals img")) > 0
                tiene_logo = len(auto.find_elements(By.CSS_SELECTOR, "img.user-avatar")) > 0
                es_automotora = "Si" if (tiene_pro or tiene_logo) else "No"

                try:
                    planes = auto.find_elements(By.CSS_SELECTOR, "div.d3-ad-tile__corner-icons div[title]")
                    plan_pagado = "; ".join([p.get_attribute("title") for p in planes])
                except:
                    plan_pagado = ""

                try:
                    precio_raw = auto.find_element(By.CSS_SELECTOR, "div.d3-ad-tile__price").text.split("\n")[0].strip()
                    precio = limpiar_numero(precio_raw)
                except:
                    precio = 0

                try:
                    detalles = auto.find_elements(By.CSS_SELECTOR, "ul.d3-ad-tile__details li")
                    anio, km = "N/A", 0
                    for d in detalles:
                        svg_use = d.find_element(By.TAG_NAME, "use").get_attribute("xlink:href")
                        if "year" in svg_use:
                            anio = d.text.strip()
                        elif "mileage" in svg_use:
                            km = limpiar_numero(d.text.strip())
                except:
                    anio, km = "N/A", 0

                try:
                    link = auto.find_element(By.CSS_SELECTOR, "a.d3-ad-tile__description").get_attribute("href")
                except:
                    link = ""

                autos_info.append({
                    "titulo": titulo,
                    "vendedor": vendedor,
                    "es_automotora": es_automotora,
                    "plan_pagado": plan_pagado,
                    "precio": precio,
                    "anio": anio,
                    "km": km,
                    "link": link,
                    "pagina": page
                })

            # Detalle
            for idx, info in enumerate(autos_info, start=1):
                ubicacion, fecha_publicacion = "N/A", "N/A"
                if info["link"]:
                    try:
                        driver.get(info["link"])
                        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.d3-property-info__header-data")))
                        header_divs = driver.find_elements(By.CSS_SELECTOR, "div.d3-property-info__header-data > div")
                        for div in header_divs:
                            text = div.text.strip()
                            if "/" in text:
                                fecha_publicacion = text
                            elif text and not text.isdigit():
                                ubicacion = text
                    except Exception as e:
                        print(f"⚠️ Error en detalle {info['link']}: {e}")

                all_rows.append({
                    "timestamp": datetime.datetime.now().isoformat(timespec="seconds"),
                    "pagina": info["pagina"],
                    "titulo_auto": info["titulo"],
                    "vendedor": info["vendedor"],
                    "es_automotora": info["es_automotora"],
                    "total_autos_vendedor": 0,
                    "plan_pagado": info["plan_pagado"],
                    "precio": info["precio"],
                    "anio": info["anio"],
                    "km": info["km"],
                    "ubicacion": ubicacion,
                    "fecha_publicacion": fecha_publicacion,
                    "link": info["link"]
                })

                print(f"✅ [{page}-{idx}] {info['titulo']} | ${info['precio']} | {info['anio']} | {info['km']} km | {ubicacion} | {fecha_publicacion}")

                # guardar en lotes
                if len(all_rows) % BATCH_SIZE == 0:
                    guardar_csv(all_rows)
                    all_rows = []

                time.sleep(0.3)

        except Exception as e:
            print(f"❌ Error en página {page}: {e}")
            guardar_csv(all_rows)
            all_rows = []
            continue

    # Guardar lo que quede al final
    if all_rows:
        guardar_csv(all_rows)

finally:
    driver.quit()

🌐 Página 1/1313: https://www.yapo.cl/autos-usados.1
✅ [1-1] Peugeot Partner | $7690000 | 2022 | 68000 km | La Florida | 07/10/2025
✅ [1-2] Chevrolet Colorado | $21980000 | 2026 | 61 km | Santiago | 07/10/2025
✅ [1-3] Nissan Versa | $10690000 | 2022 | 45500 km | Maule | 06/10/2025
✅ [1-4] Chevrolet Captiva | $13500000 | 2023 | 28913 km | Las Condes | 02/10/2025
✅ [1-5] Ford Ranger | $6990000 | 2016 | 1790000 km | San Miguel | 06/10/2025
✅ [1-6] Chevrolet Montana | $14980000 | 2024 | 7200 km | Providencia | 05/10/2025
✅ [1-7] Honda HR-V | $13980000 | 2019 | 62597 km | Las Condes | 02/10/2025
✅ [1-8] Chevrolet Cruze | $10690000 | 2018 | 76100 km | La Serena | 05/10/2025
✅ [1-9] Maxus T60 | $12900000 | 2019 | 89000 km | Concepción | 04/10/2025
✅ [1-10] Changan Alsvin | $7990000 | 2023 | 42920 km | La Serena | 05/10/2025
✅ [1-11] Kia Grand Carnival | $12990000 | 2016 | 172000 km | Vitacura | 03/10/2025
✅ [1-12] BMW 220 | $24990000 | 2022 | 74000 km | Vitacura | 03/10/2025
✅ [1-13] Renault C